In [7]:
import pandas as pd
from glob import glob
import os
import re

First, we want to clean up the hundreds of .txt files in the name subdirectory by combining them all, in order, into one big .csv file.

In [11]:
# naming convention of the name txt files for each year of birth
pattern = os.path.join('names', 'yob*.txt')
files = sorted(glob(pattern))

if not files:
    raise FileNotFoundError(f"No files found with pattern: {pattern}")

dfs = []
for path in files:
    # extract yob from file name
    fname = os.path.basename(path)
    try:
        year = int(fname.replace('yob', '').replace('.txt', ''))
    except ValueError:
        # skip files that don't match the expected pattern
        print(f"Skipping unexpected filename: {fname}")
        continue

    # read file
    df = pd.read_csv(path, header=None, names=['name', 'sex', 'count'], dtype={'name': str, 'sex': str, 'count': int})
    # create year column using yob from file names
    df['year'] = year
    dfs.append(df)

# concatenate dbs, ensure data types are correct + consistent
all_names = pd.concat(dfs, ignore_index=True)
all_names['count'] = all_names['count'].astype(int)
all_names['year'] = all_names['year'].astype(int)

# sort by year and popularity
all_names = all_names.sort_values(['year', 'count'], ascending=[True, False]).reset_index(drop=True)

# write to csv
out_path = os.path.join('names', 'all_years.csv')
all_names.to_csv(out_path, index=False)
print(f"Wrote combined CSV with {len(all_names)} rows to: {out_path}")

Wrote combined CSV with 2149477 rows to: names/all_years.csv


Now, we need to load the TV shows dataset- allshows.txt. We need to read the file, clean up column names, remove streaming services/non-US shows, and extract the year from the start time for each show.

In [12]:
# latin1 encodinng helps us avoid weird characters that sometimes show up
shows = pd.read_csv("allshows.txt", encoding="latin1")
shows.head()

,title,directory,tvrage,TVmaze,start date,end date,number of episodes,run time,network,country,onhiatus,onhiatusdesc
0,A for Andromeda,AforAndromeda,764.0,6921.0,Oct 1961,Nov 1961,7 eps,45 min,BBC,UK,False,NaN
1,ï¿½ La Carte,ALaCarte,NaN,61712.0,May 2022,___ ____,6 eps,30 min,Allblk,US,True,NaN
2,The A List,AList,NaN,37579.0,Oct 2018,Jun 2021,21 eps,30 min,BBC iPlayer,UK,False,NaN
3,A to Z,AtoZ,37968.0,92.0,Oct 2014,Jan 2015,13 eps,30 min,NBC,US,False,NaN
4,The A Word,AWord,51488.0,11402.0,Mar 2016,Jun 2020,18 eps,60 min,BBC One,UK,False,NaN


In [13]:
# store num of shows before filtering to track efficiency
before = len(shows)

# clean up column names
shows.columns = shows.columns.str.strip().str.lower().str.replace(" ", "_")

# keep only US shows
shows = shows[shows['country'] == "US"]

# remove streaming-exclusive platforms (case-insensitive)
streaming_keywords = ["netflix", "hulu", "amazon", "prime", "apple", "hbo max", "max", "paramount", "peacock", "roku", "allblk"]
shows = shows[~shows['network'].str.lower().str.contains("|".join(streaming_keywords), na=False)]

# remove invalid start dates
shows = shows[shows['start_date'].notna()]
shows = shows[~shows['start_date'].str.contains("___", na=False)]

def extract_year(date):
    # extract the last 4-digit number in a string so we can isolate just the year
    match = re.search(r"\b(19\d{2}|20\d{2})\b", str(date))
    return int(match.group()) if match else None

shows['year'] = shows['start_date'].apply(extract_year)
shows = shows.dropna(subset=['year'])
shows['year'] = shows['year'].astype(int)

# check if it worked
shows[['title', 'network', 'country', 'year']].head()

,title,network,country,year
3,A to Z,NBC,US,2014
6,Aaahh!!! Real Monsters,Nick,US,1994
7,Aaron Stone,Disney XD,US,2009
8,Aaron's Way,NBC,US,1988
9,The Abbott and Costello Show,syndicated,US,1952


In [ ]:
# attempt to filter down shows to only scripted shows
non_scripted_keywords = ["reality", "news", "documentary", "competition", "game show", "sports"]
shows = shows[~shows['directory'].str.lower().str.contains("|".join(non_scripted_keywords), na=False)]

after = len(shows)
print("Rows after filtering: ", after)
print("Rows removed: ", before - after)

Rows after filtering:  7811
Rows removed:  5244


For now, the best way we have found to measure popularity is based on how many episodes that show has. Intuitively, this makes sense- if a show is running for many years, they keep getting renewed for new seasons, meaning it is well-liked. An unpopular show will likely have less episodes. 

In [16]:
# extract the number of episodes for our popularity metric
def extract_eps(x):
    match = re.search(r"(\d+)", str(x))
    return int(match.group()) if match else None

shows['episodes'] = shows['number_of_episodes'].apply(extract_eps)
shows['episodes'] = shows['episodes'].fillna(0)

# find the most popular show each year
top_show_per_year = shows.sort_values(['year', 'episodes'], ascending=[True, False]).groupby('year').head(1)
top_show_per_year[['title', 'year', 'episodes']].head()

,title,year,episodes
6587,Lights Out (1946),1946,160.0
3368,The Ed Sullivan Show,1948,1068.0
11098,Suspense (1949),1949,260.0
4321,The George Burns and Gracie Allen Show,1950,291.0
9401,The Red Skelton Show,1951,672.0


We need to create an empty table to track the "influence" each show has had on baby names during that year. We will fill in the top 5 character names per show.

In [17]:
influence = top_show_per_year[['title', 'year']].copy()
influence['character_names'] = "" 
influence.head()

,title,year,character_names
6587,Lights Out (1946),1946,
3368,The Ed Sullivan Show,1948,
11098,Suspense (1949),1949,
4321,The George Burns and Gracie Allen Show,1950,
9401,The Red Skelton Show,1951,


Now, we need to prepare the baby name data by using within-year ranks.

In [18]:
all_names['rank'] = all_names.groupby('year')['count'].rank(ascending=False, method='dense')
all_names.head()

,name,sex,count,year,rank
0,John,M,9655,1880,1.0
1,William,M,9532,1880,2.0
2,Mary,F,7065,1880,3.0
3,James,M,5927,1880,4.0
4,Charles,M,5348,1880,5.0


Finally, we will merge each name/year row with the show of that year to join these two datasets.

In [19]:
merged = pd.merge(all_names, top_show_per_year[['year', 'title', 'episodes']], on='year', how='inner')
merged.head()

,name,sex,count,year,rank,title,episodes
0,James,M,87436,1946,1.0,Lights Out (1946),160.0
1,Robert,M,84138,1946,2.0,Lights Out (1946),160.0
2,John,M,79266,1946,3.0,Lights Out (1946),160.0
3,Mary,F,67467,1946,4.0,Lights Out (1946),160.0
4,William,M,60030,1946,5.0,Lights Out (1946),160.0
